# Clasificadores

OpenCV implementa un subconjunto de algoritmos de aprendizaje máquina que nos pueden ayudar a realizar una serie de tareas de análisis de datos. Otras librerías de python, como scikit-learn disponen de un conjunto más amplio de algoritmos de clasificación, regresión y clustering.

La resolución de un problema de clasificación consta de una fase de aprendizaje en la que el clasificador es entrenado con un conjunto de ejemplos de entrada y sus correspondientes etiquetas o salidas. Idealmente se dispone de otro conjunto de ejemplos de entradas y salidas para validar la calidad del entrenamiento realizado.

Por tanto, el paso previo a utilizar un clasificador es construir nuestro conjunto de entrenamiento. En una situación real, el conjunto de entrenamiento vendrá determinado por los vectores de características calculados a partir de la imagen y por el etiquetado manual realizado sobre dichas imágenes.

En los siguientes ejemplos utilizaremos un conjunto de entrenamiento que incluye la propia librería. El conjunto de entrenamiento está formado por 20000 vectores de características calculados a partir de imágenes con letras. La primera columna de cada fila es un caracter que representa lo que aparece en la imagen.

In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Leer el fichero con los ejemplos y convertir las etiquetas
# que aparecen en la primera columna a números
data = np.loadtxt('res/letter-recognition.data', 
                    dtype='float32', 
                    delimiter = ',',
                    converters= {0: lambda ch: ord(ch)-ord('A')})

labels = data[:,0] # Primera columna: etiquetas
#print(labels)
# Se crea un array unidimensional que es necesario convertir a bidimensional
labels = labels.reshape((data.shape[0], 1)).astype(np.int32) 
print(labels)
#data = data[:,1:] # Restantes columnas: vectores de caracteristicas

[[19]
 [ 8]
 [ 3]
 ...
 [19]
 [18]
 [ 0]]


## División del conjunto de entrenamiento

Necesitamos evaluar la calidad del entrenamiento en un conjunto de datos que no haya sido usado para entrenar. Si tenemos suficientes ejemplos, la forma más sencilla de obtener nuestro conjunto de test es reservar un subconjunto de ejemplos del conjunto de entrenamiento, preferiblemente de forma aleatoria.

Una forma aún mejor de evaluar la calidad del entrenamiento es utilizar validación cruzada. Si existe un número suficiente de ejemplos de cada clase, podemos usar K-fold y si tenemos pocos ejemplos, mejor optar por un método Leave-one-out.

Una forma de generar de forma aleatoria bloques de ejemplos para entrenamiento y test usando una kfold es la siguiente:

In [2]:
indices = np.arange(0, data.shape[0]) # Todos los indices del vector
np.random.shuffle(indices) # Aleatorizar los indices del vector
print(indices.shape, indices)

k = 10 # k de la k-fold
step = int(data.shape[0] / k) # Numero de ejemplos usados en cada fold (2000)

# En cada iteracion de la k-fold seleccionaremos un bloque distinto de
# indices para el conjunto de test y los k-1 restantes para 
# entrenamiento:
# Iteracion 1: T E E E E E E E E E 
# Iteracion 2: E T E E E E E E E E 
# Iteracion 3: E E T E E E E E E E 
# ...
# Iteracion 10: E E E E E E E E E T 

for i in range(k):
    # Indices para test
    test_idx = indices[i*step:(i+1)*step]
    print(i, "test", test_idx.shape, test_idx)
    
    # Indices para entrenamiento    
    train_idx_a = indices[:i*step] 
    train_idx_b = indices[(i+1)*step:]
    train_idx = np.concatenate([train_idx_a, train_idx_b])
    print(i, "train", train_idx.shape, train_idx)
    
    # Obtener el conjunto de entrenamiento a partir de los índices
    train_data_k = data[train_idx]
    train_labels_k = labels[train_idx]

    # Obtener el conjunto de test a partir de los índices
    test_data_k = data[test_idx]
    test_labels_k = labels[test_idx]



(20000,) [15189 19928 14200 ...  9504 13308 14918]
0 test (2000,) [15189 19928 14200 ...  9573 18495   871]
0 train (18000,) [11185  2251 11343 ...  9504 13308 14918]
1 test (2000,) [11185  2251 11343 ...  4600 16996 16993]
1 train (18000,) [15189 19928 14200 ...  9504 13308 14918]
2 test (2000,) [ 5378  3462  2955 ...   376 17634  4583]
2 train (18000,) [15189 19928 14200 ...  9504 13308 14918]
3 test (2000,) [15985 19664  7076 ... 11361 15276 15391]
3 train (18000,) [15189 19928 14200 ...  9504 13308 14918]
4 test (2000,) [ 2456 15087 14360 ...  5410  9838 13118]
4 train (18000,) [15189 19928 14200 ...  9504 13308 14918]
5 test (2000,) [ 4799 14876 18225 ...    24 17229  3099]
5 train (18000,) [15189 19928 14200 ...  9504 13308 14918]
6 test (2000,) [ 7512 11616 14783 ...  2009  2244  6747]
6 train (18000,) [15189 19928 14200 ...  9504 13308 14918]
7 test (2000,) [17609 14532  2926 ...  9835  8914  2065]
7 train (18000,) [15189 19928 14200 ...  9504 13308 14918]
8 test (2000,) [19842

## K-Nearest Neighbors

La función `cv2.ml.KNearest_create` es la implementación de OpenCV del algoritmo K-Nearest Neigbors. No necesita ningún parámetro y devuelve un objeto clasificador.

El objeto clasificador contiene el método `train` que permite entrenar el clasificador. Este método recibe los siguientes parámetros:
- Matriz con los ejemplos de entrenamiento
- Disposición de los ejemplos en la matriz (por filas o por columnas)
- Matriz con las etiquetas de los ejemplos de entrenamiento

In [6]:
# Dividimos el conjunto original de entrenamiento en entrenamiento y test

train_data = data[:18000,:]
train_labels = labels[:18000,:]
test_data = data[-2000:,:]
test_labels = labels[-2000:,:]

# Creamos el clasificador
knn = cv2.ml.KNearest_create()
# Entrenamos el clasificador
knn.train(train_data, cv2.ml.ROW_SAMPLE, train_labels)

True

Una vez el clasificador ha sido entrenado es posible utilizarlo para predecir nuevos ejemplos. Para ello se utiliza la función `findNearest`, que recibe dos parámetros: los ejemplos a predecir y el número de vecinos más cercanos que se considerarán para obtener el resultado de la clasificación. Esta función tiene cuatro parámetros de salida:
- `ret`: Clase del primer elemento del vector (sólo es útil si predecimos un único ejemplo).
- `results`: Matriz en la que cada fila contiene las predicciones del ejemplo correspondiente
- `neighbours`: Matriz en el que se muestran los k vecinos más cercanos del ejemplo a predecir
- `dist`: Distancia de cada uno de los k vecinos al ejemplo a predecir.

En este caso vamos a utilizar esta función con los ejemplos del conjunto de test:

In [4]:
# Validamos el clasificador con el conjunto de test
ret, results, neighbours, dist = knn.findNearest(test_data, 3)
# Comprobamos resultados para los tres primeros ejemplos del conjunto de test
for i in range(3):
    print("---- Ejemplo", i, "-----")
    print("Clase seleccionada", results[i])
    print("Vecinos mas proximos", neighbours[i])
    print("Distancia a vecinos", dist[i])
    print("Etiqueta real", test_labels[i])
    

---- Ejemplo 0 -----
Clase seleccionada [15.]
Vecinos mas proximos [15. 15. 15.]
Distancia a vecinos [ 7. 11. 13.]
Etiqueta real [15]
---- Ejemplo 1 -----
Clase seleccionada [12.]
Vecinos mas proximos [12. 12. 12.]
Distancia a vecinos [3. 3. 4.]
Etiqueta real [12]
---- Ejemplo 2 -----
Clase seleccionada [21.]
Vecinos mas proximos [21. 21. 21.]
Distancia a vecinos [5. 6. 8.]
Etiqueta real [21]


Cuando realizamos un entrenamiento es interesante conocer el porcentaje de aciertos obtenido en el conjunto de test, el cual nos ofrece una idea aproximada de cómo funcionará nuestro clasificador en un sistema real.

Una forma de calcular los aciertos es la siguiente:

In [5]:
# Comprobación de los resultados en el conjunto de test
# Aciertos => todos aquellos ejemplos en los que el vecino más
# cercano coincide con la etiqueta establecida
success = np.sum(results == test_labels)

print("Aciertos", success, "/", test_labels.shape[0], 100*float(success)/float(test_labels.shape[0]), "%")

Aciertos 1904 / 2000 95.2 %


### Ejercicio
Entrena un clasificador K-Nearest Neigbors con el conjunto de entrenamiento de ejemplo utilizando una K-fold con k =10 y calcula el promedio de acierto en los k conjuntos de test.

In [7]:
# Se toma como punto de partida la creación y entrenamiento del clasificador con el conjunto de entrenamiento bajo una K-fold con k=10:

# Creación del clasificador
knn = cv2.ml.KNearest_create()

# Entrenamiento del clasificador
knn.train(train_data_k, cv2.ml.ROW_SAMPLE, train_labels_k)

# Se valida el clasificador con el conjunto de test estableciendo 3 vecinos más cercanos considerados:
ret, results, neighbours, dist = knn.findNearest(test_data_k, 3)

# Al igual que en el ejemplo mostrado, se proporcionan los resultados para los tres primeros ejemplos del conjunto de test:
for i in range(3):
    print("---- Ejemplo", i, "-----")
    print("Clase seleccionada", results[i])
    print("Vecinos mas proximos", neighbours[i])
    print("Distancia a vecinos", dist[i])
    print("Etiqueta real", test_labels_k[i])

# Se procede al cálculo de promedio de acierto en los k conjuntos de test:
success = np.sum(results == test_labels_k)
print("Aciertos en los k conjuntos de test", success, "/", test_labels_k.shape[0], 100*float(success)/float(test_labels_k.shape[0]), "%")

---- Ejemplo 0 -----
Clase seleccionada [0.]
Vecinos mas proximos [0. 0. 0.]
Distancia a vecinos [4. 6. 6.]
Etiqueta real [0]
---- Ejemplo 1 -----
Clase seleccionada [21.]
Vecinos mas proximos [21. 21. 21.]
Distancia a vecinos [0. 0. 3.]
Etiqueta real [21]
---- Ejemplo 2 -----
Clase seleccionada [15.]
Vecinos mas proximos [15. 15. 15.]
Distancia a vecinos [ 7. 10. 11.]
Etiqueta real [15]
Aciertos en los k conjuntos de test 1922 / 2000 96.1 %


## Random Trees

La función `cv2.RTrees_create` permite crear un clasificador de tipo Random Trees. El formato de la función de entrenamiento `train` es exactamente el mismo que en K-Nearest Neighbors. Sin embargo, la función para predecir las clases es `predict`, que toma como único parámetro la matriz con los ejemplos de predicción. Esta función devuelve dos salidas:
- `ret`: Clase del primer elemento del vector (sólo es útil si predecimos un único ejemplo).
- `results`: Matriz en la que cada fila contiene las predicciones del ejemplo correspondiente.

In [8]:
rt = cv2.ml.RTrees_create()
rt.setMaxDepth(100)

rt.train(train_data, cv2.ml.ROW_SAMPLE, train_labels)
ret, results = rt.predict(test_data)

success = np.sum(results == test_labels)

print("Aciertos", success, "/", test_labels.shape[0],100*float(success)/float(test_labels.shape[0]), "%")


Aciertos 1870 / 2000 93.5 %


### Ejercicio
Entrena un clasificador Random Forest con el conjunto de entrenamiento de ejemplo utilizando una K-fold con k=10 y calcula el promedio de acierto en los k conjuntos de test.

In [9]:
# Se crea el clasificador Random Forest con denominación rt2:
rt2 = cv2.ml.RTrees_create()
rt2.setMaxDepth(100)

# Entrenamiento del clasificador:
rt2.train(train_data_k, cv2.ml.ROW_SAMPLE, train_labels_k)
ret2, results2 = rt2.predict(test_data_k)

# Cálculo del promedio de acierto en los k conjuntos de test:
success2 = np.sum(results2 == test_labels_k)

print("Aciertos de rt2 en los k conjuntos de test", success2, "/", test_labels_k.shape[0],100*float(success2)/float(test_labels_k.shape[0]), "%")

Aciertos de rt2 en los k conjuntos de test 1891 / 2000 94.55 %


## SVM
La función `cv2.SVM_create` sirve para crear un clasificador de tipo SVM. El formato de las funciones `train` y `predict` es exactamente el mismo que en Random Forest. 

In [22]:
svm = cv2.ml.SVM_create()
svm.train(train_data, cv2.ml.ROW_SAMPLE, train_labels)
ret, results = svm.predict(test_data)

success = np.sum(results == test_labels)

print("Aciertos", success, "/", test_labels.shape[0], 100*float(success)/float(test_labels.shape[0]), "%")

Aciertos 1024 / 2000 51.2 %


El entrenamiento se realiza utilizando los parámetros por defecto pero no siempre estos parámetros obtienen los mejores resultados de clasificación. Probar distintos parámetros puede ser una labor tediosa por lo que OpenCV cuenta con un método que entrena el algoritmo con distintos parámetros y devuelve el objeto entrenado con los parámetros óptimos. Este método utiliza una k-fold internamente durante la optimización. Nota: puede tardar varios minutos en ejecutarse la operación!

In [23]:
svm.trainAuto(train_data, cv2.ml.ROW_SAMPLE, train_labels)

True

### Ejercicio
Calcula el porcentaje de acierto en el conjunto de test con el svm entrenado optimizando parámetros.

In [3]:
# Se crea el clasificador de tipo svm con denominación "svm_ejercicio" y parámetros optimizados:
svm_ejercicio = cv2.ml.SVM_create()
svm_ejercicio.trainAuto(train_data_k, cv2.ml.ROW_SAMPLE, train_labels_k)
ret_ejercicio, results_ejercicio = svm_ejercicio.predict(test_data_k)

# Cálculo del procentaje de acierto:
success_ejercicio = np.sum(results_ejercicio == test_labels_k)
print("Aciertos con el clasificador svm con parámetros optimizados", success_ejercicio, "/", test_labels_k.shape[0], 100*float(success_ejercicio)/float(test_labels_k.shape[0]), "%")


Aciertos con el clasificador svm con parámetros optimizados 1966 / 2000 98.3 %


## Guardar y cargar clasificadores
Cuando trabajamos en un problema real en un problema que requiera un clasificador, el modo de operación habitual es entrenar uno o varios clasificadores con validación cruzada y tratando de optimizar parámetros para obtener las tasas de acierto más elevadas. El mejor clasificador será el que se utilice para tareas de predicción.

Para evitar reentrenar el clasificador cada vez que se necesite realizar una predicción, el clasificador se puede salvar a un fichero y cargarlo cuando sea necesario utilizarlo. Para ello, todos los objetos creados con `XXXX_create` disponen de los métodos `save` y `load`. Ambos métodos toman como parámetro el nombre del fichero donde se almacenará o desde el que se cargará el clasificador. El método `load` devuelve además el clasificador entrenado.


In [10]:
svm_ejercicio.save('svm-trained-characters.xml')

svm2 = cv2.ml.SVM_create()
svm2 = svm2.load('svm-trained-characters.xml')


rt.save('rt-trained-characters.xml')

rt2 = cv2.ml.RTrees_create()
rt2 = rt2.load('rt-trained-characters.xml')
